In [103]:
import pandas as pd
from io import StringIO
import requests as r
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np
import plotly.graph_objects as go

Criei uma função para fazer DataFrames a partir de um url

In [104]:
def faz_df(url):
    req = r.request('GET', url, headers={'user-agent': 'Mozilla/5.0'})
    df = pd.read_json(StringIO(req.text), encoding='UTF-8')
    return df

Criando os DataFrames

In [105]:
information_df = faz_df('https://treinamento-ndados.s3.sa-east-1.amazonaws.com/boardgames_information_raw.json')
categoricals_df = faz_df('https://treinamento-ndados.s3.sa-east-1.amazonaws.com/boardgames_categoricals_raw.json')
stat_df = faz_df('https://treinamento-ndados.s3.sa-east-1.amazonaws.com/boardgames_stat_raw.json')

Juntando os DataFrames em um só, com o nome de 'merged'

In [106]:
pd.set_option('display.max_columns', 500)
merged = pd.concat(objs=[information_df, categoricals_df, stat_df], axis=1)
merged.head(3)

,object_id,name,yearpublished,sortindex,minplayers,maxplayers,minplaytime,maxplaytime,minage,objectid,label,boardgamedesigner_cnt,boardgameartist_cnt,boardgamepublisher_cnt,boardgamehonor_cnt,boardgamecategory_cnt,boardgamemechanic_cnt,boardgameexpansion_cnt,boardgameversion_cnt,boardgamefamily_cnt,boardgamedesigner,boardgameartist,boardgamepublisher,boardgamehonor,boardgamecategory,boardgameversion,boardgamemechanic,boardgameexpansion,boardgamefamily,gamelink,objectid,min_community,max_community,totalvotes,playerage,languagedependence,usersrated,average,baverage,stddev,avgweight,numweights,numgeeklists,numtrading,numwanting,numcomments,siteviews,numplays,numplays_month,news,blogs,weblink,podcast
0,174430,Gloomhaven,2017,1,1,4,60,120,12,174430,Board Game,1,3,9,23,5,12,4,19,7,"['Isaac Childres""']","['Alexandr Elichev', 'Josh T. McDowell', 'Alva...","['Cephalofair Games', 'Albi', 'Asmodee', 'Feue...",['2017 Best Science Fiction or Fantasy Board G...,"['Adventure', 'Exploration', 'Fantasy', 'Fight...","['Chinese edition', 'Czech edition', 'English ...","['Campaign / Battle Card Driven', 'Cooperative...","['Gloomhaven: Forgotten Circles', 'Gloomhaven:...","['Campaign Games', 'Components: Miniatures', '...",/boardgame/174430/gloomhaven,174430,3.0,4.0,827.0,14,4,31254.0,8.85292,8.58424,1.59819,3.8078,1311,3657,313,1365,5972,8933078,230213,3478,7,471,31,139
1,161936,Pandemic Legacy Season 1,2015,2,2,4,60,60,13,161936,Board Game,2,1,11,20,2,8,0,33,3,"['Rob Daviau""', 'Matt Leacock""']",['Chris Quilliams'],"['Z-Man Games', 'Asterion Press', 'Devir', 'Fi...",['2015 Cardboard Republic Immersionist Laurel ...,"['Environmental', 'Medical']","['Chinese blue edition', 'Chinese red edition'...","['Action Points', 'Cooperative Game', 'Hand Ma...",['None'],"['Campaign Games', 'Legacy', 'Pandemic']",/boardgame/161936/pandemic-legacy-season-1,161936,4.0,4.0,549.0,12,4,34729.0,8.62499,8.47159,1.59463,2.8301,971,3612,272,771,5477,2971746,196621,1090,9,597,69,164
2,167791,Terraforming Mars,2016,3,1,5,120,120,12,167791,Board Game,1,1,20,20,6,9,15,29,6,"['Jacob Fryxelius""']",['Isaac Fryxelius'],"['FryxGames', 'Arclight', 'Fantasmagoria', 'Gh...",['2016 Cardboard Republic Architect Laurel Nom...,"['Economic', 'Environmental', 'Industry / Manu...","['Bulgarian edition', 'Chinese edition', 'Czec...","['Card Drafting', 'End Game Bonuses', 'Hand Ma...",['French Championship Promo Cards (fan expansi...,"['Fryxgames Future Timeline', 'Planets: Mars',...",/boardgame/167791/terraforming-mars,167791,3.0,4.0,1252.0,12,3,48339.0,8.42299,8.26781,1.36938,3.2313,1863,5941,277,2068,7274,4724387,245997,4380,14,1158,60,148


In [107]:
print(stat_df.shape)
print(information_df.shape)
print(categoricals_df.shape)
print(merged.shape)

(20016, 23)
(20016, 9)
(20016, 21)
(20016, 53)


Vemos que o merged contem a soma de colunas dos outros 3 dataframes. Agora, vou remover algumas colunas consideradas inúteis.

In [108]:
merged.drop(labels=['objectid', 'label'], axis=1, inplace=True)
merged.shape

(20016, 50)

Fazendo um heatmap das variáveis estatísticas:


In [109]:
# corr = stat_df.corr(method='pearson')

# plt.figure(figsize=(15,15))
# sns.heatmap(corr, annot=True, fmt='.2f', cmap='coolwarm')
# plt.show()

## Medindo a popularidade de um jogo:

A tabela estatística contém diversos dados que, teoricamente, medem a popularidade de um jogo. Entretanto, por enquanto, considerei apenas as médias de avaliação do jogo (BGG e dos jogadores) e a quantidade de avaliações. Posteriormente, há espaço para aplicar algum modelo de clusterização para analisarmos todas essas variáveis juntas, mas por enquanto prevalece o modelo simplificado.

In [110]:
colunas_selecionadas = ['name', 'object_id', 'usersrated', 'average', 'baverage', 'numgeeklists',
                        'numwanting', 'numcomments', 'siteviews', 'news', 'blogs', 'weblink',
                        'podcast']
boardgame_stat = merged[colunas_selecionadas].copy()
stat_df_filtrado = boardgame_stat.copy()
stat_df_filtrado

,name,object_id,usersrated,average,baverage,numgeeklists,numwanting,numcomments,siteviews,news,blogs,weblink,podcast
0,Gloomhaven,174430,31254.0,8.85292,8.58424,3657,1365,5972,8933078,7,471,31,139
1,Pandemic Legacy Season 1,161936,34729.0,8.62499,8.47159,3612,771,5477,2971746,9,597,69,164
2,Terraforming Mars,167791,48339.0,8.42299,8.26781,5941,2068,7274,4724387,14,1158,60,148
3,Through the Ages A New Story of Civilization,182028,18269.0,8.49419,8.23513,2083,1049,2660,2448074,5,185,30,42
4,Brass Birmingham,224517,10070.0,8.62031,8.20459,1039,1077,1702,894621,3,124,13,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20011,Tuppi,14765,33.0,7.18788,5.53873,15,2,15,8992,0,0,0,0
20012,History of War,8134,96.0,5.99240,5.51421,56,4,50,43202,0,0,2,0
20013,Hesketh's Legacy,2673,1.0,6.00000,0.00000,2,0,1,2559,0,2,0,0
20014,Snug as a Bug in a Rug,132322,42.0,6.36905,5.52454,13,1,17,9524,0,0,4,0


Aqui encontrei um problema: há diversos dados nulos nas colunas baverage e average, e não sei o que essa nulicidade representa diretamente. Portanto, vou prever esses valores com base na razão entre average e baverage. A seguir fica o passo a passo de como fiz para o cálculo da razão.

## Análise Prévia

### Analisando usersrated == 0


In [111]:
stat_df_filtrado.query('usersrated == 0').sum()

name            Looney LeoRolazoneContangoDon't Give Up Your D...
object_id                                                  360634
usersrated                                                    0.0
average                                                       0.0
baverage                                                      0.0
numgeeklists                                                  550
numwanting                                                     62
numcomments                                                    35
siteviews                                                  438442
news                                                            0
blogs                                                           1
weblink                                                        12
podcast                                                         0
dtype: object

Vou comparar os valores de cada variável que contém userrated == 0 com o valor total delas, para saber sua representatividade.

In [112]:
def verifica_porcentagem(df:pd.DataFrame, coluna):
    valor = df.query('usersrated == 0').sum()[coluna]
    total = df[coluna].sum()
    return valor / total * 100

In [113]:
colunas_analisadas = ['numgeeklists', 'numwanting', 'numcomments', 'siteviews', 'blogs', 'weblink']
for coluna in colunas_analisadas:
    print(f"A porcentagem de {coluna} é {verifica_porcentagem(stat_df_filtrado, coluna)}")

A porcentagem de numgeeklists é 0.011253797901023462
A porcentagem de numwanting é 0.007868988949147294
A porcentagem de numcomments é 0.0009131586156515387
A porcentagem de siteviews é 0.02870729860782448
A porcentagem de blogs é 0.0004923682914820286
A porcentagem de weblink é 0.009585812996764788


Como todos os valores representam nem 1% do total, serão desconsiderados.

In [114]:
# Removi todas as linhas nas quais usersrated == 0
stat_df_filtrado = stat_df_filtrado.query('usersrated != 0').copy()

In [115]:
stat_df_filtrado.head(3)

,name,object_id,usersrated,average,baverage,numgeeklists,numwanting,numcomments,siteviews,news,blogs,weblink,podcast
0,Gloomhaven,174430,31254.0,8.85292,8.58424,3657,1365,5972,8933078,7,471,31,139
1,Pandemic Legacy Season 1,161936,34729.0,8.62499,8.47159,3612,771,5477,2971746,9,597,69,164
2,Terraforming Mars,167791,48339.0,8.42299,8.26781,5941,2068,7274,4724387,14,1158,60,148


### Analisando average == 0

In [116]:
stat_df_filtrado.query('average == 0').sum()

name              0
object_id         0
usersrated      0.0
average         0.0
baverage        0.0
numgeeklists      0
numwanting        0
numcomments       0
siteviews         0
news              0
blogs             0
weblink           0
podcast           0
dtype: object

In [117]:
stat_df_filtrado.query('average == 0').shape

(0, 13)

Não há linhas nas quais o average é 0, então não há nada a se fazer

### Analisando baverage == 0

In [118]:
stat_df_filtrado.query('baverage == 0').sum()

name            The Garden GamePeaceful ResistanceGreen Thumb ...
object_id                                                 4223416
usersrated                                                15670.0
average                                                6768.32561
baverage                                                      0.0
numgeeklists                                                22960
numwanting                                                   3063
numcomments                                                  9586
siteviews                                                10745815
news                                                            4
blogs                                                          88
weblink                                                       972
podcast                                                        18
dtype: object

Aparentemente, temos algumas linhas com baverage == 0

In [119]:
stat_df_filtrado.query('baverage == 0').shape

(1296, 13)

São 1296 linhas nas quais baverage == 0. Vamos ver o quanto isso é em relação aos votos totais.

### Observando a ocorrencia de baverage == 0 em relação aos votos totais

In [120]:
print(f"A porcentagem de usersrated com baverage == 0 por total é {stat_df_filtrado.query('baverage == 0')['usersrated'].sum() / stat_df_filtrado['usersrated'].sum() * 100}")

A porcentagem de usersrated com baverage == 0 por total é 0.10620128769908506


Como as ocorrencias de baverage == 0 refletem 0.1% dos votos todais, optei por desonsiderá-las para efeito de cálculo

In [121]:
stat_df_filtrado = stat_df_filtrado.query('baverage != 0').copy()

### Lidando com dados nulos:

In [122]:
stat_df_filtrado.isnull().sum()

name             0
object_id        0
usersrated      89
average         94
baverage        96
numgeeklists     0
numwanting       0
numcomments      0
siteviews        0
news             0
blogs            0
weblink          0
podcast          0
dtype: int64

Como os dados nulos ocasionarão em futuros erros ou novos dados nulos após efetuação do cálculo, desconsideraremos

In [123]:
stat_df_filtrado = stat_df_filtrado.dropna().copy()

In [124]:
stat_df_filtrado.isnull().sum()

name            0
object_id       0
usersrated      0
average         0
baverage        0
numgeeklists    0
numwanting      0
numcomments     0
siteviews       0
news            0
blogs           0
weblink         0
podcast         0
dtype: int64

## Cálculo da razão

#### Efetuando cálculo da razão geral

In [125]:
stat_df_filtrado['average/baverage'] = stat_df_filtrado['average'] / stat_df_filtrado['baverage']

#### Ponderando o efeito da razão pela quantidade total de votos

In [126]:
stat_df_filtrado['average/baverage_ponderado'] =stat_df_filtrado['average/baverage'] * stat_df_filtrado['usersrated']

#### Efetuando o cálculo da razão média ponderada de average / baverage

In [127]:
razao_media = stat_df_filtrado['average/baverage_ponderado'].sum() / stat_df_filtrado['usersrated'].sum()

In [128]:
razao_media

1.0617524552384836

## Remapeando na base original

#### Removendo registros sem votos:

In [129]:
boardgame_stat = boardgame_stat.query('usersrated != 0').copy()

#### Tratando dados nulos e zerados como o mesmo problema

In [130]:
boardgame_stat = boardgame_stat.fillna(0)
boardgame_stat

,name,object_id,usersrated,average,baverage,numgeeklists,numwanting,numcomments,siteviews,news,blogs,weblink,podcast
0,Gloomhaven,174430,31254.0,8.85292,8.58424,3657,1365,5972,8933078,7,471,31,139
1,Pandemic Legacy Season 1,161936,34729.0,8.62499,8.47159,3612,771,5477,2971746,9,597,69,164
2,Terraforming Mars,167791,48339.0,8.42299,8.26781,5941,2068,7274,4724387,14,1158,60,148
3,Through the Ages A New Story of Civilization,182028,18269.0,8.49419,8.23513,2083,1049,2660,2448074,5,185,30,42
4,Brass Birmingham,224517,10070.0,8.62031,8.20459,1039,1077,1702,894621,3,124,13,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20011,Tuppi,14765,33.0,7.18788,5.53873,15,2,15,8992,0,0,0,0
20012,History of War,8134,96.0,5.99240,5.51421,56,4,50,43202,0,0,2,0
20013,Hesketh's Legacy,2673,1.0,6.00000,0.00000,2,0,1,2559,0,2,0,0
20014,Snug as a Bug in a Rug,132322,42.0,6.36905,5.52454,13,1,17,9524,0,0,4,0


In [131]:
boardgame_stat.query('average == 0 | baverage == 0')

,name,object_id,usersrated,average,baverage,numgeeklists,numwanting,numcomments,siteviews,news,blogs,weblink,podcast
41,Le Havre,35677,24266.0,7.87980,0.00000,5739,1554,4868,1876534,3,253,67,30
64,Battlestar Galactica The Board Game,37111,31058.0,0.00000,7.58289,10878,756,6426,2220403,0,307,100,94
170,Deception Murder in Hong Kong,156129,13172.0,7.51194,0.00000,1484,710,2337,748461,2,132,19,44
209,\u05e7\u05d9\u05e0\u05d2\u05d3\u05d5\u05de\u05...,204583,24282.0,0.00000,7.22535,2295,346,3644,824994,7,322,66,59
218,Sid Meier's Civilization The Board Game,77130,13748.0,7.43552,0.00000,3559,405,2501,1475436,1,92,66,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19999,Obsession,5163,5.0,4.60000,0.00000,18,1,4,3806,0,0,0,0
20006,Cham\u00e4leon,3104,22.0,6.56818,0.00000,7,1,11,5627,0,0,0,0
20009,Welcome to the Dungeon,150312,10331.0,0.00000,6.49553,1340,150,1782,433329,7,118,42,16
20010,Supremacy,5073,6.0,6.66667,0.00000,2,1,1,8206,0,0,0,0


#### Removendo duplo 0 (average == 0 e baverage == 0)

In [132]:
boardgame_stat['average+baverage'] = boardgame_stat['average'] + boardgame_stat['baverage']

In [133]:
boardgame_stat = boardgame_stat.query('average+baverage != 0').copy()

#### Efetuando o mapeamento

In [134]:
boardgame_stat.head()

,name,object_id,usersrated,average,baverage,numgeeklists,numwanting,numcomments,siteviews,news,blogs,weblink,podcast,average+baverage
0,Gloomhaven,174430,31254.0,8.85292,8.58424,3657,1365,5972,8933078,7,471,31,139,17.43716
1,Pandemic Legacy Season 1,161936,34729.0,8.62499,8.47159,3612,771,5477,2971746,9,597,69,164,17.09658
2,Terraforming Mars,167791,48339.0,8.42299,8.26781,5941,2068,7274,4724387,14,1158,60,148,16.69080
3,Through the Ages A New Story of Civilization,182028,18269.0,8.49419,8.23513,2083,1049,2660,2448074,5,185,30,42,16.72932
4,Brass Birmingham,224517,10070.0,8.62031,8.20459,1039,1077,1702,894621,3,124,13,21,16.82490


In [135]:
# Defini duas funções para tratar os dados zerados(que antes eram nulos) das colunas
# average e baverage

def tratamento_average(linha):
    if linha['average'] <= 0.1:
        return linha['baverage'] * razao_media
    else:
        return linha['average']
    
def tratamento_baverage(linha):
    if linha['baverage'] <= 0.1:
        return linha['average'] / razao_media
    else:
        return linha['baverage']

In [136]:
# Criando uma coluna nova para os novos(e antigos) valores average
boardgame_stat['average_tratado'] = boardgame_stat.apply(tratamento_average, axis=1)

In [137]:
# Criando uma coluna nova para os novos(e antigos) valores baverage
boardgame_stat['baverage_tratado'] = boardgame_stat.apply(tratamento_baverage, axis=1)

## Validação

#### Teste 1

In [138]:
# objectid == 35677 era um dos valores de baverage que eu sabia que estava nulo
boardgame_stat.query('object_id == 35677')['baverage_tratado']

41    7.421504
Name: baverage_tratado, dtype: float64

In [139]:
boardgame_stat.query('object_id == 35677')['average']

41    7.8798
Name: average, dtype: float64

In [140]:
boardgame_stat.query('object_id == 35677')['average'] / razao_media

41    7.421504
Name: average, dtype: float64

#### Teste 2

In [141]:
# Esse objectid é como o outro, já sabia que estava zerado, só que agora na coluna average
boardgame_stat.query('object_id == 37111')['average_tratado']

64    8.051152
Name: average_tratado, dtype: float64

In [142]:
boardgame_stat.query('object_id == 37111')['baverage']

64    7.58289
Name: baverage, dtype: float64

In [143]:
boardgame_stat.query('object_id == 37111')['baverage'] * razao_media

64    8.051152
Name: baverage, dtype: float64

In [144]:
boardgame_stat.head()

,name,object_id,usersrated,average,baverage,numgeeklists,numwanting,numcomments,siteviews,news,blogs,weblink,podcast,average+baverage,average_tratado,baverage_tratado
0,Gloomhaven,174430,31254.0,8.85292,8.58424,3657,1365,5972,8933078,7,471,31,139,17.43716,8.85292,8.58424
1,Pandemic Legacy Season 1,161936,34729.0,8.62499,8.47159,3612,771,5477,2971746,9,597,69,164,17.09658,8.62499,8.47159
2,Terraforming Mars,167791,48339.0,8.42299,8.26781,5941,2068,7274,4724387,14,1158,60,148,16.69080,8.42299,8.26781
3,Through the Ages A New Story of Civilization,182028,18269.0,8.49419,8.23513,2083,1049,2660,2448074,5,185,30,42,16.72932,8.49419,8.23513
4,Brass Birmingham,224517,10070.0,8.62031,8.20459,1039,1077,1702,894621,3,124,13,21,16.82490,8.62031,8.20459


# Machine Learning

In [145]:
colunas_clusterizadas = ['usersrated', 'numgeeklists', 'numwanting', 'numcomments', 
                         'siteviews', 'news', 'blogs', 'weblink', 'podcast', 'average_tratado',
                          'baverage_tratado']
X = boardgame_stat[colunas_clusterizadas].copy()

#### Usarei o algoritmo de clusterização Kmeans

In [146]:
from sklearn.cluster import KMeans
from sklearn import metrics

In [147]:
# Utilziando o algoritmo e percorrendo diferentes valores de k
valores_k = []
inercias = []

for i in range(1,10):
    kmeans = KMeans(n_clusters = i, random_state=0, n_init='auto') .fit(X)
    valores_k.append(i)
    inercias.append(kmeans.inertia_)

# Dar uma olhada em tipos de init.

In [148]:
# Visualizando a relação entre inércia e K
fig_cotovelo = px.line(x=valores_k, y=inercias
).update_layout(xaxis_title='Quantidade de clusters', yaxis_title='Inércias')
                                                                       
fig_cotovelo.show()

In [149]:
# Verificando o coeficiente de silhueta para os calores de k
valores_ks = []
s = []

for i in range(2,10):
    kmeans = KMeans(n_clusters=i, random_state=0, n_init='auto').fit(X)
    valores_ks.append(i)
    s.append(metrics.silhouette_score(X, kmeans.labels_))

In [150]:
fig_silhueta = px.line(x=valores_ks, y=s).update_layout(xaxis_title='Quantidade de clusters',
                                                        yaxis_title='Coeficiente de silhueta')
fig_silhueta.show()

# Modelo ajustado: Clusterização dos dados estatísticos

In [151]:
# Aplicando o algoritmo 
model_stat = KMeans(n_clusters=5, random_state=0, n_init='auto')
df_clusterizado = boardgame_stat[colunas_clusterizadas].copy()
labels_stat = model_stat.fit_predict(df_clusterizado)
df_clusterizado['Cluster'] = labels_stat

Descrição dos clusters:


In [152]:
# Função que agrupa o dataframe clusterizado e cria a coluna com a quantidade total de itens em cada cluster
def descricao(df: pd.DataFrame, grupamento: str) -> pd.DataFrame:
    descricao = df.groupby(grupamento)
    n = descricao.size()
    descricao = descricao.mean()
    descricao['Quantidade Total'] = n
    
    return descricao

In [153]:
descricao(df_clusterizado, 'Cluster')

,usersrated,numgeeklists,numwanting,numcomments,siteviews,news,blogs,weblink,podcast,average_tratado,baverage_tratado,Quantidade Total
Cluster,,,,,,,,,,,,
0,222.489819,108.769222,16.329698,76.928563,3.276307e+04,0.339769,3.768610,4.216090,0.701458,6.216063,5.553135,17974
1,10955.879365,2629.755556,480.533333,2351.187302,8.368732e+05,3.612698,130.158730,41.850794,24.307937,7.381666,7.011794,315
2,39694.172414,7448.620690,956.241379,7306.068966,3.858579e+06,8.896552,526.068966,94.206897,100.137931,7.864858,7.666282,29
3,2873.803818,1025.134957,166.907176,766.382488,2.793917e+05,1.615537,39.330481,18.776169,7.949967,7.022250,6.332230,1519
4,22617.531646,4172.139241,780.000000,4212.658228,1.745673e+06,5.278481,245.050633,62.379747,46.126582,7.610453,7.345280,79


### Descrevendo cada cluster

Visando explicar o motivo de um cluster ser melhor que outro, irei criar duas novas colunas que mostram uma nota de 0 a 10 de visibilidade de um jogo. Para isso, utilizarei o método de normalização MinMax.

In [154]:
from sklearn.preprocessing import MinMaxScaler

Para melhor descrever as estatísticas que refletem a popularidade de um jogo fora do site, vou juntar as colunas news, blogs, weblink e podcast em uma chamada 'visibilidade externa'.


In [155]:
df_clusterizado['visibilidade externa'] = df_clusterizado['news'] + df_clusterizado['blogs'] + df_clusterizado['weblink'] + df_clusterizado['podcast']

In [188]:
df_clusterizado['visibilidade externa'] = MinMaxScaler().fit_transform(np.array(df_clusterizado['visibilidade externa']).reshape(-1,1))
df_clusterizado['visibilidade externa'] = df_clusterizado['visibilidade externa'] * 10
df_clusterizado.sort_values(by=['visibilidade externa'], ascending=False).head()
df_clusterizado['visibilidade externa'].describe()

count    19916.000000
mean         0.118467
std          0.342764
min          0.000000
25%          0.012500
50%          0.037500
75%          0.093750
max         10.000000
Name: visibilidade externa, dtype: float64

Para as features que refletem as estatíticas do prórpio site, irei separa-las em outro DataFrame para a normalização.

In [157]:
df_clusterizado_visibilidade_interna = df_clusterizado[['numgeeklists', 'numwanting', 'numcomments', 'siteviews', 'usersrated']]

In [183]:

arr_scaled = MinMaxScaler().fit_transform(df_clusterizado_visibilidade_interna)
df_clusterizado_visibilidade_interna = pd.DataFrame(arr_scaled, columns=df_clusterizado_visibilidade_interna.columns, index=df_clusterizado_visibilidade_interna.index)
df_clusterizado_visibilidade_interna['visibilidade interna'] = (df_clusterizado_visibilidade_interna['numgeeklists']*0.2 + df_clusterizado_visibilidade_interna['numwanting']*0.2 
                                                                + df_clusterizado_visibilidade_interna['numcomments']*0.1 + df_clusterizado_visibilidade_interna['siteviews']*0.1 
                                                                + df_clusterizado_visibilidade_interna['usersrated']*0.4)
df_clusterizado_visibilidade_interna['visibilidade interna'] = df_clusterizado_visibilidade_interna['visibilidade interna'] * 10
df_clusterizado_visibilidade_interna.sort_values(by=['visibilidade interna'], ascending=False)

,numgeeklists,numwanting,numcomments,siteviews,usersrated,visibilidade interna
160,0.125937,0.246132,0.989383,0.367141,0.996032,6.084791
352,0.121071,0.216151,1.000000,0.390841,1.000000,6.065284
82,0.115240,0.299807,0.873593,0.383491,0.987336,6.036520
48,0.092146,0.489362,0.735461,0.381848,0.821338,5.565675
2,0.049001,1.000000,0.424313,0.528855,0.532779,5.182284
...,...,...,...,...,...,...
19603,0.000000,0.000000,0.000058,0.000263,0.000011,0.000366
19693,0.000025,0.000000,0.000000,0.000217,0.000022,0.000355
19786,0.000016,0.000000,0.000058,0.000218,0.000011,0.000353
19931,0.000016,0.000000,0.000000,0.000231,0.000011,0.000308


In [184]:
df_clusterizado['visibilidade interna'] = df_clusterizado_visibilidade_interna['visibilidade interna'].copy()

In [185]:
clusters = df_clusterizado['Cluster'].unique()
agrupamento_cluster = df_clusterizado.groupby(by=['Cluster']).mean()
agrupamento_cluster

,usersrated,numgeeklists,numwanting,numcomments,siteviews,news,blogs,weblink,podcast,average_tratado,baverage_tratado,visibilidade externa,visibilidade interna
Cluster,,,,,,,,,,,,,
0,222.489819,108.769222,16.329698,76.928563,3.276307e+04,0.339769,3.768610,4.216090,0.701458,6.216063,5.553135,0.056412,0.035531
1,10955.879365,2629.755556,480.533333,2351.187302,8.368732e+05,3.612698,130.158730,41.850794,24.307937,7.381666,7.011794,1.249563,1.221938
2,39694.172414,7448.620690,956.241379,7306.068966,3.858579e+06,8.896552,526.068966,94.206897,100.137931,7.864858,7.666282,4.558190,3.655775
3,2873.803818,1025.134957,166.907176,766.382488,2.793917e+05,1.615537,39.330481,18.776169,7.949967,7.022250,6.332230,0.422951,0.380988
4,22617.531646,4172.139241,780.000000,4212.658228,1.745673e+06,5.278481,245.050633,62.379747,46.126582,7.610453,7.345280,2.242722,2.261447


In [186]:
fig_visibilidade = go.Figure(data=[
    go.Bar(name='visibilidade interna', x=agrupamento_cluster.index, y=agrupamento_cluster['visibilidade interna']),
    go.Bar(name='visibilidade externa', x=agrupamento_cluster.index, y=agrupamento_cluster['visibilidade externa'])
]).update_layout(barmode='group')
fig_visibilidade.show()

Vemos, portanto, que os clusters 2 e 4 são os que possuem maior visibilidade no geral. Nesse sentido, iremos analisar quais categorias dos jogos pertencentes aos clusters 2 e 4 fazem com que eles sejam os mais populares.

### Visualização:

In [162]:
# Vamos visualizar os clusters de acordo com as seguintes caracteristicas: Quantidade de avaliações(usersrated), visualizações no site(siteviews)
# e nota (baverage_tratado)

fig_cluster = px.scatter_3d(df_clusterizado,x='usersrated', y='siteviews', z='baverage_tratado', color='Cluster')
fig_cluster.show()

## Análise de categoria 

#### Ajustando DataFrames:

In [163]:
# Adicionando a coluna Name e ObjectId ao DataFrame clusterizado
df_clusterizado_categ = df_clusterizado.copy()
df_clusterizado_categ['name'] = boardgame_stat['name']
df_clusterizado_categ['objectid'] = boardgame_stat['object_id']

Aqui, quis juntar o DataFrame clusterizado com o DataFrame das categorias com base nos valores que estão no DataFrame clusterizado. Entretando, notei que o DataFrame das categorias possui diversos valores dulpicados, alguns iguais em todas as colunas e outros iguais somente em algumas. Dito isso, utilizei o drop_duplicates com base no objectid, esse método vai apagar a duplicata que está num indice maior. Apesar de, aparentemente, perdermos parte dos dados, haviam apenas 3 casos nos quais o valor do objectid era igual, mas o restante das colunas diferente, então acredito que não houve perda.

In [164]:
categoricals_df.drop_duplicates(inplace=True, subset=['objectid'])

In [165]:
# Adicionei as categorias ao DataFrame de estatísticas que continha os clusters
df_final = df_clusterizado_categ.merge(categoricals_df, how='left', on='objectid')
print(df_final.shape)
print(df_clusterizado.shape)

(19916, 36)
(19916, 14)


In [166]:
df_final.query('Cluster == 2').shape

(29, 36)

In [167]:
df_final.query('Cluster == 2').head()

,usersrated,numgeeklists,numwanting,numcomments,siteviews,news,blogs,weblink,podcast,average_tratado,baverage_tratado,Cluster,visibilidade externa,visibilidade interna,name,objectid,label,boardgamedesigner_cnt,boardgameartist_cnt,boardgamepublisher_cnt,boardgamehonor_cnt,boardgamecategory_cnt,boardgamemechanic_cnt,boardgameexpansion_cnt,boardgameversion_cnt,boardgamefamily_cnt,boardgamedesigner,boardgameartist,boardgamepublisher,boardgamehonor,boardgamecategory,boardgameversion,boardgamemechanic,boardgameexpansion,boardgamefamily,gamelink
0,31254.0,3657,1365,5972,8933078,7,471,31,139,8.85292,8.58424,2,4.05000,3.949540,Gloomhaven,174430,Board Game,1,3,9,23,5,12,4,19,7,"['Isaac Childres""']","['Alexandr Elichev', 'Josh T. McDowell', 'Alva...","['Cephalofair Games', 'Albi', 'Asmodee', 'Feue...",['2017 Best Science Fiction or Fantasy Board G...,"['Adventure', 'Exploration', 'Fantasy', 'Fight...","['Chinese edition', 'Czech edition', 'English ...","['Campaign / Battle Card Driven', 'Cooperative...","['Gloomhaven: Forgotten Circles', 'Gloomhaven:...","['Campaign Games', 'Components: Miniatures', '...",/boardgame/174430/gloomhaven
1,34729.0,3612,771,5477,2971746,9,597,69,164,8.62499,8.47159,2,5.24375,2.811975,Pandemic Legacy Season 1,161936,Board Game,2,1,11,20,2,8,0,33,3,"['Rob Daviau""', 'Matt Leacock""']",['Chris Quilliams'],"['Z-Man Games', 'Asterion Press', 'Devir', 'Fi...",['2015 Cardboard Republic Immersionist Laurel ...,"['Environmental', 'Medical']","['Chinese blue edition', 'Chinese red edition'...","['Action Points', 'Cooperative Game', 'Hand Ma...",['None'],"['Campaign Games', 'Legacy', 'Pandemic']",/boardgame/161936/pandemic-legacy-season-1
2,48339.0,5941,2068,7274,4724387,14,1158,60,148,8.42299,8.26781,2,8.62500,4.940395,Terraforming Mars,167791,Board Game,1,1,20,20,6,9,15,29,6,"['Jacob Fryxelius""']",['Isaac Fryxelius'],"['FryxGames', 'Arclight', 'Fantasmagoria', 'Gh...",['2016 Cardboard Republic Architect Laurel Nom...,"['Economic', 'Environmental', 'Industry / Manu...","['Bulgarian edition', 'Chinese edition', 'Czec...","['Card Drafting', 'End Game Bonuses', 'Hand Ma...",['French Championship Promo Cards (fan expansi...,"['Fryxgames Future Timeline', 'Planets: Mars',...",/boardgame/167791/terraforming-mars
6,36644.0,9658,1341,8095,4506683,2,323,84,82,8.31307,8.16138,2,3.06875,3.886318,Twilight Struggle,12333,Board Game,2,5,12,13,3,9,9,32,5,"['Ananda Gupta""', 'Jason Matthews""']","['Viktor Csete', 'Rodger B. MacGowan', 'Chechu...","['GMT Games', '(Self-Published)', 'Asterion Pr...",['2005 Charles S. Roberts Best Modern Era Boar...,"['Modern Warfare', 'Political', 'Wargame']","['Bard Centrum Gier Polish deluxe edition', 'B...","['Action/Event', 'Advantage Token', 'Area Majo...",['Twilight Struggle: Anni di Piombo Promo Card...,"['Cold War', 'Country: Soviet Union', 'Country...",/boardgame/12333/twilight-struggle
7,18991.0,1956,1367,3037,2949888,1,204,35,65,8.42602,8.15575,2,1.90625,2.602343,Star Wars Rebellion,187645,Board Game,1,47,9,12,5,6,1,10,2,"['Corey Konieczka""']","['Matt Allsopp', 'David Ardila', 'Balaskas', '...","['Fantasy Flight Games', 'ADC Blackfire Entert...",['2016 Best Science Fiction or Fantasy Board G...,"['Fighting', 'Miniatures', 'Movies / TV / Radi...","['Czech edition', 'English edition', 'French e...","['Area Majority / Influence', 'Area Movement',...",['Star Wars: Rebellion u2013 Rise of the Empire'],"['Components: Miniatures', 'Star Wars']",/boardgame/187645/star-wars-rebellion


df_final indica a junção do DataFrame de categorias filtrado com os clusters junto do DataFrame de categorias. Por conta disso, vou remover algumas colunas que considero inúteis do DataFrame de categorias.

In [168]:
df_final.drop(labels=['boardgamedesigner_cnt', 'boardgameartist_cnt', 'boardgamepublisher_cnt', 'boardgamehonor_cnt', 'boardgamecategory_cnt', 'boardgamemechanic_cnt', 'boardgameexpansion_cnt'
                      , 'boardgameversion_cnt', 'boardgamefamily_cnt', 'label', 'boardgamedesigner', 'boardgameartist', 'boardgamepublisher', 'boardgamehonor', 'boardgameversion'
                      , 'boardgameexpansion', 'gamelink'], inplace=True, axis=1)

In [169]:
# Aqui transformei o tipo dos valores dessas colunas de str para listas
df_final['boardgamecategory'] = df_final.boardgamecategory.apply(lambda x: x[1:-1].split(','))
df_final['boardgamemechanic'] = df_final.boardgamemechanic.apply(lambda x: x[1:-1].split(','))
df_final['boardgamefamily'] = df_final.boardgamefamily.apply(lambda x: x[1:-1].split(','))


In [170]:
# Defini uma função para formatar a lista que foi transformada na célula anterior
def formata_lista(lista_teste):
    lista_formatada = []
    for elemento in lista_teste:
        elemento_formatado = elemento.strip().replace("'", "")
        lista_formatada.append(elemento_formatado)
    return lista_formatada

In [171]:
# Formatando as listas de cada colunas
df_final['boardgamecategory'] = df_final.boardgamecategory.apply(formata_lista)
df_final['boardgamemechanic'] = df_final.boardgamemechanic.apply(formata_lista)
df_final['boardgamefamily'] = df_final.boardgamefamily.apply(formata_lista)

## Analisando cada cluster:

Aqui vamos bater os clusters com suas informações categóricas, para assim concluir que categoria, familia e mecanica os jogos mais populares possuem. Como dito anteriormente, os clusters 2 e 4 representam os jogos mais famosos, 

### Clusters 2 e 4

In [172]:
# Criei um novo DataFrame com as linhas que estão no cluster 2
df_final_cluster2 = df_final.query('Cluster == 2').copy()
df_final_cluster4 = df_final.query('Cluster == 4').copy()
df_final_cluster2.head()

,usersrated,numgeeklists,numwanting,numcomments,siteviews,news,blogs,weblink,podcast,average_tratado,baverage_tratado,Cluster,visibilidade externa,visibilidade interna,name,objectid,boardgamecategory,boardgamemechanic,boardgamefamily
0,31254.0,3657,1365,5972,8933078,7,471,31,139,8.85292,8.58424,2,4.05000,3.949540,Gloomhaven,174430,"[Adventure, Exploration, Fantasy, Fighting, Mi...","[Campaign / Battle Card Driven, Cooperative Ga...","[Campaign Games, Components: Miniatures, Crowd..."
1,34729.0,3612,771,5477,2971746,9,597,69,164,8.62499,8.47159,2,5.24375,2.811975,Pandemic Legacy Season 1,161936,"[Environmental, Medical]","[Action Points, Cooperative Game, Hand Managem...","[Campaign Games, Legacy, Pandemic]"
2,48339.0,5941,2068,7274,4724387,14,1158,60,148,8.42299,8.26781,2,8.62500,4.940395,Terraforming Mars,167791,"[Economic, Environmental, Industry / Manufactu...","[Card Drafting, End Game Bonuses, Hand Managem...","[Fryxgames Future Timeline, Planets: Mars, Sol..."
6,36644.0,9658,1341,8095,4506683,2,323,84,82,8.31307,8.16138,2,3.06875,3.886318,Twilight Struggle,12333,"[Modern Warfare, Political, Wargame]","[Action/Event, Advantage Token, Area Majority ...","[Cold War, Country: Soviet Union, Country: USA..."
7,18991.0,1956,1367,3037,2949888,1,204,35,65,8.42602,8.15575,2,1.90625,2.602343,Star Wars Rebellion,187645,"[Fighting, Miniatures, Movies / TV / Radio the...","[Area Majority / Influence, Area Movement, Dic...","[Components: Miniatures, Star Wars]"


Como cada jogo pode ter mais de uma mecanica e pertencer a mais de uma categoria/familia, defini uma função que conta quantas aparições cada valor unico de mecanica, categoria e familia aparecem. Nesse sentido, poderemos analisar cada feature individualmente e extrair quais são mais populares.

In [173]:
# Função que retorna um data frame com a contagem que cada feature unico de um DataFrame
def conta_feature(df: pd.DataFrame, coluna) -> pd.DataFrame:
    count_df = {}
    for lista in df[coluna]:
        for valor in lista:
            count_df[valor] = count_df.get(valor, 0) + 1
    count_df = pd.DataFrame.from_dict(count_df, orient='index', columns=['contagem'])
    count_df.sort_values(by=['contagem'], ascending=False, inplace=True)
    count_df.reset_index(names='feature', inplace=True)
    return count_df

# Função que retorna a porcentagem de cada item de uma coluna de um DataFrame
def cria_coluna_porcentagem(df: pd.DataFrame, coluna) -> pd.DataFrame:
    soma = df[coluna].sum()
    df['porcentagem'] = df[coluna] / soma * 100
    return df

# Função que retorna quantos valores foram percorridos até a soma das linhas de determinada coluna chegue no valor máximo 
def conta_porcentagem(df: pd.DataFrame, coluna: str, maximo: float) -> int:
    soma_porcentagem = 0
    contador = 0
    while soma_porcentagem <= maximo:
        soma_porcentagem += df[coluna][contador]
        contador += 1
    return contador + 1

#### Categorias mais presentes:

In [174]:
# Criando dois DataFrames, um com a contagem do cluster 2, o outro do 4
category_count2 = conta_feature(df_final_cluster2, 'boardgamecategory')
category_count4 = conta_feature(df_final_cluster4, 'boardgamecategory')
# Criando a coluna de porcentagem da contagem
category_count2 = cria_coluna_porcentagem(category_count2, 'contagem')
category_count4 = cria_coluna_porcentagem(category_count4, 'contagem')
categorias = category_count2['feature'].copy()

In [175]:
fig_categoria = go.Figure(data=[
    go.Bar(name='cluster 2', x=categorias, y=category_count2['porcentagem']),
    go.Bar(name='cluster 4', x=categorias, y=category_count4['porcentagem'])
]).update_layout(barmode='group')
fig_categoria.show()

In [176]:
print('As', conta_porcentagem(category_count2, 'porcentagem', 50), 'primeiras categorias representam 50% da contagem total do cluster 2')
print('As', conta_porcentagem(category_count4, 'porcentagem', 50), 'primeiras categorias representam 50% da contagem total do cluster 4')

As 9 primeiras categorias representam 50% da contagem total do cluster 2
As 11 primeiras categorias representam 50% da contagem total do cluster 4


#### Mecanicas mais presentes

In [177]:
# Criando dois DataFrames, um com a contagem do cluster 2, o outro do 4
mechanic_count2 = conta_feature(df_final_cluster2, 'boardgamemechanic')
mechanic_count4 = conta_feature(df_final_cluster4, 'boardgamemechanic')
# Criando a coluna de porcentagem da contagem
mechanic_count2 = cria_coluna_porcentagem(mechanic_count2, 'contagem')
mechanic_count4 = cria_coluna_porcentagem(mechanic_count4, 'contagem')
mecanicas = mechanic_count2['feature'].head(12).copy()

In [178]:
fig_mecanica = go.Figure(data=[
    go.Bar(name='cluster 2', x=mecanicas, y=mechanic_count2['porcentagem']),
    go.Bar(name='cluster 4', x=mecanicas, y=mechanic_count4['porcentagem'])
]).update_layout(barmode='group')
fig_mecanica.show()

In [179]:
print('As', conta_porcentagem(mechanic_count2, 'porcentagem', 50), 'primeiras mecanicas representam 50% da contagem total do cluster 2')
print('As', conta_porcentagem(mechanic_count4, 'porcentagem', 50), 'primeiras mecanicas representam 50% da contagem total do cluster 4')

As 12 primeiras mecanicas representam 50% da contagem total do cluster 2
As 11 primeiras mecanicas representam 50% da contagem total do cluster 4


#### Famílias mais presentes

In [180]:
# Criando dois DataFrames, um com a contagem do cluster 2, o outro do 4
family_count2 = conta_feature(df_final_cluster2, 'boardgamefamily')
family_count4 = conta_feature(df_final_cluster4, 'boardgamefamily')
# Criando a coluna de porcentagem da contagem
family_count2 = cria_coluna_porcentagem(family_count2, 'contagem')
family_count4 = cria_coluna_porcentagem(family_count4, 'contagem')
familias = family_count2['feature'].head(31).copy()

In [181]:
fig_familia = go.Figure(data=[
    go.Bar(name='cluster 2', x=familias, y=family_count2['porcentagem']),
    go.Bar(name='cluster 4', x=familias, y=family_count4['porcentagem'])
]).update_layout(barmode='group')
fig_familia.show()

In [182]:
print('As', conta_porcentagem(family_count2, 'porcentagem', 50), 'primeiras familias representam 50% da contagem total do cluster 2')
print('As', conta_porcentagem(family_count4, 'porcentagem', 50), 'primeiras familias representam 50% da contagem total do cluster 4')

As 15 primeiras familias representam 50% da contagem total do cluster 2
As 31 primeiras familias representam 50% da contagem total do cluster 4


### Agrupando variáveis

Percebemos que a base de dados contém muitas variáveis que medem, teoricamente, a popularidade e qualidade de um jogo. Nesse sentido, vamos agrupa-las para termos uma melhor visão sobre os dados.

O primeiro agrupamento fará referencia a todas as estatísticas que estão fora do site, e será chamado de visibilidade exterma. Nessa nova colunas, estarão as contagens de citações em notícias, blogs, sites e podcasts, somadas. 

In [ ]:
df['weblink2'] = (df['weblink'] - min(df['weblink'])) / (max(df['weblink']) - min(df['weblink']))
df['blogs2'] = (df['blogs'] - min(df['blogs'])) / (max(df['blogs']) - min(df['blogs']))
df['podcast2'] = (df['podcast'] - min(df['podcast'])) / (max(df['podcast']) - min(df['podcast']))
df['news2'] = (df['news'] - min(df['news'])) / (max(df['news']) - min(df['news']))
df['visibilidade_externa'] = df['weblink2'] + df['blogs2'] + df['podcast2'] + df['news2']
df['visibilidade_externa2'] = ((df['visibilidade_externa'] - min(df['visibilidade_externa'])) / (max(df['visibilidade_externa']) - min(df['visibilidade_externa']))) * 10
df.head()

O segundo agrupamento será o conjunto de estatísticas que trazem dados do próprio site. Nele estarão contidas os valores normalizados pela média das colunas: Quantidade de avaliações, número de adições nas geeklists, número de pessoas que querem o jogo, número de comentários de um jogo, número de visitações na página no jogo.

In [ ]:
df['usersrated2'] = (df['usersrated'] - min(df['usersrated'])) / (max(df['usersrated']) - min(df['usersrated']))
df['numgeeklists2'] = (df['numgeeklists'] - min(df['numgeeklists'])) / (max(df['numgeeklists']) - min(df['numgeeklists']))
df['numwanting2'] = (df['numwanting'] - min(df['numwanting'])) / (max(df['numwanting']) - min(df['numwanting']))
df['numcomments2'] = (df['numcomments'] - min(df['numcomments'])) / (max(df['numcomments']) - min(df['numcomments']))
df['siteviews2'] = (df['siteviews'] - min(df['siteviews'])) / (max(df['siteviews']) - min(df['siteviews']))
df['visibilidade_interna'] = df['usersrated2'] + df['numgeeklists2'] + df['numwanting2'] + df['numcomments2'] + df['siteviews2']
df['visibilidade_interna2'] = ((df['visibilidade_interna'] - min(df['visibilidade_interna'])) / (max(df['visibilidade_interna']) - min(df['visibilidade_interna']))) * 10
df.head()

df_filtrado_cluster2e4['age_class2'] = pd.cut(
    df_filtrado_cluster2e4['minage'], bins=[0, 6, 11, 14, df_filtrado_cluster2e4['minage'].max()],
      labels=['early_child', 'grown_child', 'teenager', 'adult']
    )